In [1]:
import pickle
import numpy as np

with open('../data/qm9_local_envs.pkl', 'rb') as f:
    local_envs = pickle.load(f)

local_envs = [np.array(local_env) for local_env in local_envs]

In [6]:
from tqdm import tqdm

min_bond_length_ratio = np.inf
for env in tqdm(local_envs):
    bond_lengths = np.linalg.norm(env, axis=1)
    bond_length_ratio = min(bond_lengths) / max(bond_lengths)
    min_bond_length_ratio = min(min_bond_length_ratio, bond_length_ratio)

print(min_bond_length_ratio)

100%|██████████| 1177631/1177631 [00:03<00:00, 354364.67it/s]

0.5995928845261392


In [11]:
from tqdm import tqdm
import numpy as np

min_angle_degrees = 180.0  # Initialize with maximum possible angle
for env in tqdm(local_envs):
    # Skip environments with fewer than 2 points
    if env.shape[0] < 2:
        continue
    
    # Normalize all vectors (points)
    norms = np.linalg.norm(env, axis=1, keepdims=True)
    normalized_env = env / norms
    
    # Calculate all pairwise dot products
    dot_products = np.dot(normalized_env, normalized_env.T)
    
    # Create a mask for the upper triangle (excluding the diagonal)
    mask = np.triu(np.ones((env.shape[0], env.shape[0]), dtype=bool), k=1)
    
    # Find the maximum dot product (which corresponds to the minimum angle)
    max_dot_product = np.max(dot_products[mask])
    
    # Calculate the minimum angle in this environment (in degrees)
    # Clip to handle numerical precision issues
    clipped_dot_product = np.clip(max_dot_product, -1.0, 1.0)
    env_min_angle = np.degrees(np.arccos(clipped_dot_product))
    
    # Update the overall minimum angle
    min_angle_degrees = min(min_angle_degrees, env_min_angle)

print(min_angle_degrees)

100%|██████████| 1177631/1177631 [00:16<00:00, 70261.77it/s]

43.87487115065688
